In [ ]:
import os
from os.path import isdir, join
from pathlib import Path
import pandas as pd

# Math
import numpy as np
import scipy.stats
from scipy.fftpack import fft
from scipy import signal
from scipy.io import wavfile
import librosa
import librosa.display
from scipy import sparse, stats, spatial
import scipy.sparse.linalg

from sklearn.decomposition import PCA

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import IPython.display as ipd

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import pandas as pd

%matplotlib inline

In [ ]:
N = 10

train_audio_path = '../Project/data/train/audio'
dirs = [f for f in os.listdir(train_audio_path) if isdir(join(train_audio_path, f))]
dirs.sort()

paths = []

for direct in dirs:
    if not direct.startswith('_background_noise_'):
        waves = (list(np.random.choice([train_audio_path + '/' + direct + '/' + f for f in os.listdir(join(train_audio_path, direct)) if f.endswith('.wav')],N)))
        paths.extend(waves) 

In [ ]:
N_MFCC = 20

def compute_mfcc(filepath):
    audio, sampling_rate = librosa.load(filepath, sr=None, mono=True)
    return librosa.feature.mfcc(y=audio,sr=sampling_rate)

In [ ]:
stat_name= ['mean','std','skew','kurtosis','median']
col_names = [('mfcc',stat_name[i],j) for i in range(len(stat_name))  for j in range(N_MFCC)]
features_og = pd.DataFrame(columns=pd.MultiIndex.from_tuples(col_names))

for file in paths:
    mfcc = compute_mfcc(file)
    features_og.at[file, ('mfcc', 'mean')] = np.mean(mfcc,axis=1)
    features_og.at[file, ('mfcc', 'std')] = np.std(mfcc,axis=1)
    features_og.at[file, ('mfcc', 'skew')] = scipy.stats.skew(mfcc,axis=1)
    features_og.at[file, ('mfcc', 'kurtosis')] = scipy.stats.kurtosis(mfcc,axis=1)
    features_og.at[file, ('mfcc', 'median')] = np.median(mfcc,axis=1)

features_og

In [ ]:
# drop features
features = pd.DataFrame(features_og)
features -= features.mean(axis=0)
features /= features.std(axis=0)

distances = spatial.distance.squareform(spatial.distance.pdist(features,'cosine'))

n=distances.shape[0]
kernel_width = distances.mean()
weights = np.exp(np.divide(-np.square(distances),kernel_width**2))
np.fill_diagonal(weights,0)

In [ ]:
fix, axes = plt.subplots(2, 2, figsize=(17, 8))
def plot(weights, axes):
    axes[0].spy(weights)
    axes[1].hist(weights[weights > 0].reshape(-1), bins=50);
plot(weights, axes[:, 0])

NEIGHBORS = 30

for i in range(weights.shape[0]):
    idx = weights[i,:].argsort()[:-NEIGHBORS]
    weights[i,idx] = 0
    weights[idx,i] = 0

plot(weights, axes[:, 1])
   

In [ ]:
degrees = np.sum(weights,axis=0)
laplacian = np.diag(degrees**-0.5) @ (np.diag(degrees) - weights) @ np.diag(degrees**-0.5)
laplacian = sparse.csr_matrix(laplacian)
plt.matshow(laplacian.todense());

In [ ]:
eigenvalues, eigenvectors = sparse.linalg.eigsh(A=laplacian,k=10,which='SM')

plt.plot(eigenvalues, '.-', markersize=15);

x = eigenvectors[:,1]
y = eigenvectors[:,2]

In [ ]:
fix, axes = plt.subplots(5, 5, figsize=(17, 8))
for i in range(1,6):
    for j in range(1,6):
        x = eigenvectors[:,i]
        y = eigenvectors[:,j]
        labels = np.sign(x)
        axes[i-1,j-1].scatter(x, y, c=labels, cmap='RdBu', alpha=0.5)